In [1]:
# import os
# from pathlib import Path
# from google.colab import drive

# PATH = "/content/drive"
# drive.mount(PATH)

# folder_path = Path('MyDrive/models/abs_bert')

In [2]:
# !pip install --quiet transformers datasets
# !pip install --quiet tensorboard
# tensorboard --logdir=runs

In [3]:
import torch
device = 'cuda:7' if torch.cuda.is_available() else 'cpu'
torch.cuda.is_available()

True

In [4]:
from collections import defaultdict
import os
import datetime
from pathlib import Path
import json
from tqdm import tqdm

from torch import nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler

# import datasets
# from datasets import load_dataset
from transformers import AutoTokenizer

# import utils.load_corpus as load_corpus
# import utils.losses as l

import model.absolute_bert_fix_negative_layer_norm as Abs_bert

import importlib

In [5]:
# colab
# folder_path = Path(folder_path)
# os.makedirs(PATH/folder_path, exist_ok=True)

# 59
PATH = "data"
folder_path = Path(f"limanet")
os.makedirs(PATH/folder_path, exist_ok=True)

In [6]:
tokenizer_type = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)
text = ["I love bird.", "Hi, I'm Bob."]
tokenized = tokenizer(text, return_tensors='pt', padding=True)
tokenized['input_ids']

tensor([[ 101, 1045, 2293, 4743, 1012,  102,    0,    0,    0],
        [ 101, 7632, 1010, 1045, 1005, 1049, 3960, 1012,  102]])

In [14]:
%%time

batch_size = 128
masking_probability = .15
max_length = 256
shuffle = True
# cache_dir = PATH/Path('MyDrive/data')
cache_dir='~/data1-0756727/cache/huggingface'

from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

def load(tokenizer, masking_probability=0.15, cache_dir=None, max_length=512):
  dataset = load_dataset("wikipedia", "20220301.en", cache_dir=cache_dir)
  # dataset = load_dataset("bookcorpus", cache_dir=cache_dir)

  collator = DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=masking_probability)

  def transform(batch):
    tokenized = tokenizer(batch['text'], return_tensors='pt', padding=True, add_special_tokens=False, truncation=True, max_length=max_length)
    # print(tokenized
    input_ids, labels = collator.torch_mask_tokens(tokenized['input_ids'])

    return tokenized | {
      'input_ids': input_ids,
      'labels': labels
    }

  dataset.set_transform(transform)

  return dataset

class Trainloader:
  def __init__(self, dataset, batch_size):
    self.data = dataset
    self.batch_size = batch_size

  def __len__(self):
    return len(self.data) // batch_size + 1

  def __iter__(self):
    current_start = 0
    for i in range(self.__len__()):
      yield self.data[current_start:current_start+self.batch_size]
      current_start += self.batch_size

# import importlib
# load_corpus = importlib.reload(load_corpus)

# dataset = load_corpus.Load_bookcorpus(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')
# dataset = load_corpus.load_msmarco(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')
# dataset = load_corpus.Load_wiki(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')

# dataset.transformer(masking_probability=masking_probability)
# print(dataset.dataset['train'][:2])

dataset = load(tokenizer, masking_probability=masking_probability, cache_dir=cache_dir, max_length=max_length)

train_loader = Trainloader(dataset['train'].shuffle() if shuffle else dataset['train'], batch_size)
n = next(iter(train_loader))
# print(tokenizer.decode(n['data']))
{key: n[key].shape for key in n}

/home/adam/miniconda3/envs/lima/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


CPU times: user 3.38 s, sys: 506 ms, total: 3.89 s
Wall time: 6.23 s


{'input_ids': torch.Size([128, 256]),
 'token_type_ids': torch.Size([128, 256]),
 'attention_mask': torch.Size([128, 256]),
 'labels': torch.Size([128, 256])}

In [15]:
import numpy as np

class Sampled_softmax_cross_entropy(nn.Module):
  """https://douglasorr.github.io/2021-10-training-objectives/3-sampled/article.html"""

  def __init__(self, model, num_sampling=100):
    super().__init__()
    self.num_sampling = num_sampling
    self.model = model

  def forward(self, predictions, labels):
    """
    preds: shape [batch_size, dim]
    labels: shape [batch_size, dim]
    """
    # batch_sizes = predictions.shape[:-1]

    # model = ...  # returns (batch_size x embedding_size)
    projection = self.model.word_embeddings()  # shape (n_classes x embedding_size)
    n_classes = projection.shape[0]

    # 2. Get target label scores, paired_inner_product(pred_emb, label_emb)
    label_scores = (predictions * projection[labels, :]).sum(-1) + self.model.bias[labels]

    # 3. Sample shared noise & get scores
    samples = torch.randint(high=n_classes, size=[self.num_sampling]).to(labels.device)
    noise_scores = predictions @ projection[samples, :].T + self.model.bias[None, samples]
    noise_scores += np.log(n_classes - 1)

    # 4. Reject samples matching target label & correct for remaining samples
    reject_samples = (labels[..., None] == samples[None, :]) & (labels[..., None] != -100)  #後面是 collator 會把非預測目標填為 -100
    noise_scores -= 1e6 * reject_samples
    noise_scores -= torch.log((self.num_sampling - reject_samples.sum(-1, keepdims=True)).float())

    # 5. Apply regular softmax cross entropy
    scores = torch.cat([label_scores[..., None], noise_scores], dim=-1)
    pseudo_label = torch.masked_fill(labels.clone(), labels != -100, 0).view(-1)
    loss = torch.nn.functional.cross_entropy(scores.view(-1, scores.shape[-1]), pseudo_label)

    return loss

In [16]:
learning_rate = 0.01 # for cross entropy
learning_rate = 2e-5 # for fine tuning

model_params = {
  'depth': 8,
  'num_heads': 8,
  'dim': 512,
  # 'hidden_dim': 32,
  # 'embedding_initialize_method': 'rand',
  # 'attention_type': Absolute_global_attention,
  # 'time_dim': 64,
}
loss_params = {
  'sampling_word_size': 100,
}

# Abs_bert = importlib.reload(Abs_bert)
# l = importlib.reload(l)

# model = RNN_base(rotator.Rotator(tokenizer.vocab_size, **model_params), weight_tying=False).to(device)
# using_loss = l.Complex_mse()
# using_loss = l.Complex_triplet_loss(model)
# using_loss = l.Complex_mse_triplet_loss(model)
# using_loss = l.Complex_mse_squared_triplet_loss(model)
# using_loss = l.Mse_multiplet_loss(model.model, **loss_params)
# using_loss = l.Cos_multiplet_loss(model.model, **loss_params)

model = Abs_bert.Absolute_bert_for_masked_LM(tokenizer.vocab_size, **model_params).to(device)
# using_loss = l.Sampled_softmax_cross_entropy(model, num_sampling=loss_params['sampling_word_size'])
using_loss = Sampled_softmax_cross_entropy(model, num_sampling=loss_params['sampling_word_size'])

# model = RNN_base(rotator.Rotator(tokenizer.vocab_size, **model_params)).to(device)
# using_loss = nn.CrossEntropyLoss()
# using_loss = l.Cross_entropy_l2_embedding(model.model, min_squared_norm=model.model.dim)
# using_loss = l.Cross_entropy_l2_embedding(model.model, l2_squared_weight=1e-3)

# subpath = '20240624.12:08:14' # 4 ssm_ce_with_bias
# batch_num = 25000
# model = torch.load(PATH/folder_path/subpath/f'batch_{batch_num}-model.pt', map_location=device)

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=1e-5)
scheduler = lr_scheduler.LinearLR(optimizer, total_iters=10, start_factor=1)
# scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, )
optimizer


AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-05
    foreach: None
    fused: None
    initial_lr: 2e-05
    lr: 2e-05
    maximize: False
    weight_decay: 0.01
)

In [11]:
time_string = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=8))).strftime('%Y%m%d.%H:%M:%S')
subfolder_path = Path(time_string)

os.makedirs(PATH/folder_path/subfolder_path, exist_ok=True)

with open(PATH/folder_path/subfolder_path/f'parameters.json', 'w') as f:
  f.write(json.dumps({
    'tokenizer_type': tokenizer_type,
    'model': str(model),
    'model_params': str(model_params),
    'learning_rate': learning_rate,
    'batch_size': batch_size,
    'masking_probability': masking_probability,
    'shuffle': shuffle,
    'loss_type': str(using_loss),
    'loss_params': loss_params,
  }))
time_string

'20240711.03:00:33'

In [17]:
from torch.utils.tensorboard import SummaryWriter

# Writer will output to ./runs/ directory by default
writer = SummaryWriter(log_dir=PATH/folder_path/subfolder_path/'tensorboard')

In [ ]:
num_epochs = 3
save_every_n_batches = 5000
clip_loss = 50

multiloss = 0


print(time_string)

# os.makedirs(PATH/folder_path/subfolder_path, exist_ok=True)

##############################################################################################################
model.train() ################################################################################################
# model.eval()
##############################################################################################################

global_step = 0

loss_history = []
for epoch_num, epoch in enumerate(range(num_epochs)):
  bar = tqdm(train_loader)

  for batch_num, batch in enumerate(bar):

    optimizer.zero_grad()

    batch = {key: batch[key].to(device) for key in batch}
    predicts, targets = model(**batch)

    # predicts = predicts @ model.word_embeddings().T + model.bias
    # predicts = predicts.view(-1, tokenizer.vocab_size)
    # targets = targets.view(-1)

    if multiloss:
      losses = using_loss(predicts, targets)
      loss = sum(losses.values())
    else:
      loss = using_loss(predicts, targets)

    if clip_loss is not None:
      final_loss = torch.clip(loss, max=clip_loss)
    else:
      final_loss = loss

    loss_dict = {
      'loss': f"{loss.item(): .6f}",
      'final_loss': f"{final_loss.item(): .6f}",
    }

    if global_step == 0:
      writer.add_scalar('Loss/loss', loss, global_step)
    
    if multiloss:
      loss_dict |= {key: f"{losses[key].item(): .6f}" for key in losses}

    bar.set_postfix(loss_dict)

    # with torch.autograd.detect_anomaly(True):
      # loss.backward()
      # torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=10, norm_type=2)

    final_loss.backward()
    optimizer.step()
    global_step += 1
    
    if (batch_num % 100 == 0):
      # torch.cuda.empty_cache()
      loss_history.append([epoch_num, batch_num, loss_dict])
      # print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, batch {batch_num}")

    if (global_step % 100 == 0):
      writer.add_scalar('Loss/loss', loss, global_step)

    if (batch_num % 1000 == 999):
      scheduler.step()

    if (batch_num % save_every_n_batches == 0):
      prefix = f"epoch_{epoch_num}-batch_{batch_num}-"
      torch.save(model, PATH/folder_path/subfolder_path/(prefix + f'model.pt'))

      with open(PATH/folder_path/subfolder_path/f'epoch_{epoch_num}-history.json', 'w') as f:
        f.write(json.dumps(loss_history))

      # torch.cuda.empty_cache()

    if (batch_num in [500, 1000, 1500, 2000, 2500, 5000, 7500, 12500, 17500]):
        prefix = f"epoch_{epoch_num}-batch_{batch_num}-"
        torch.save(model, PATH/folder_path/subfolder_path/(prefix + 'model.pt'))

        with open(PATH/folder_path/subfolder_path/f'epoch_{epoch_num}-history.json', 'w') as f:
          f.write(json.dumps(loss_history))

    # del batch, loss

20240711.03:00:33


  5%|▍         | 2332/50459 [14:02<5:23:23,  2.48it/s, loss=6.367382, final_loss=6.367382]  